In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import time 

import pymorphy2
import time

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from string import punctuation
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet, stopwords

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [2]:
stopword = stopwords.words('russian') + [a for a in punctuation]
stopword

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [10]:
morph = pymorphy2.MorphAnalyzer()

In [11]:
df = pd.read_excel("Общий файл отзывов 00.xlsx")
df=df[pd.isnull(df["Статья"])!=True]
df.reset_index(inplace=True) #Удаляем старую индексацию
del df['index']

df

,Статья
0,Выражаю особую благодарность сотрудникам Сберб...
1,Выражаем большую благодарность консультанту Еф...
2,На днях получил смс с предложением открыть кре...
3,У моей свекрови Поляковой Людмилы Ивановны 02....
4,Добрый день! Мою кредитную карту 4279 0100 267...
5,Смс подтверждения покупки/продажи приходят чер...
6,"Уже полнейшим безобразием является ситуация, в..."
7,"Добрый день, уважаемые представители ПАО Сберб..."
8,"Вчера,06.02.2020 года около 20 часов 5 минут,в..."
9,14 сентября 2019 года мной был полностью погаш...


In [13]:
%%time

df2 = pd.DataFrame(columns=['Статья'])

for i in range(0, len(df)):
    s = df["Статья"][i]

    words = nltk.word_tokenize(s) #Токенизация 
    
    #print(words)
    
    words2 = [word.lower() for word in words if word not in stopword] #Стоп-слова
    for w2 in words2:
        if re.findall(r'\d', w2) or '.' in w2:
            words2.remove(w2)
    #print(words2)
    df2.loc[i]=' '.join(words2)
    
    #print(i)
    if i%1000==0:
        print(i)
        
    #print()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
Wall time: 23min 25s


In [14]:
%%time
df3 = pd.DataFrame(columns=['Статья'])

for i in range(0, len(df2)):
    
    st = df2["Статья"][i]
    st = re.sub(r"[0123456789!\'\.()№`$#,!@*:*?<>\"“”=/\«|»]", " ", st)
    while re.search("  ",st):
        st = st.replace("  "," ")
    
    df3.loc[i]= st
    
    if i%1000==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
Wall time: 16min 37s


In [15]:
%%time
df4 = pd.DataFrame(columns=['Статья'])

for i in range(0, len(df)):
    words = nltk.word_tokenize(df3["Статья"][i])
    
    for ii in range(len(words)):
        words[ii] = morph.parse(words[ii])[0].normal_form
    df4.loc[i]=' '.join(words)  

    if i%1000==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
Wall time: 1h 33min 20s


In [16]:
%%time

df5 = pd.DataFrame(columns=['Статья'])

for i in range(0, len(df)):
    wordss = nltk.word_tokenize(df4["Статья"][i])
    #print(wordss)
    
    words2 = [word.lower() for word in wordss if word not in stopword] #Стоп-слова
    for w2 in words2:
        if re.findall(r'\d', w2) or '.' in w2:
            words2.remove(w2)
        if len(w2)<2:
            words2.remove(w2)
            
    #print(words2)
    df5.loc[i]=' '.join(words2)
    
    if i%1000==0:
        print(i)

    #print()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
Wall time: 21min 15s


In [17]:
%%time
df5.reset_index(inplace=True) #Удаляем старую индексацию
del df5['index']
df5.to_excel('Общий файл отзывов_5.xlsx') #Сохраняем информацию в первичный файл 

Wall time: 2min 48s


In [18]:
df5

,Статья
0,выражать особый благодарность сотрудник сберба...
1,выражать большой благодарность консультант ефи...
2,день получить смс предложение открыть кредитка...
3,свекровь поляков людмила иван год кредит сберб...
4,добрый день кредитный карта заблокировать хотя...
5,смс подтверждение покупка продажа приходить ми...
6,полный безобразие являться ситуация который по...
7,добрый день уважаемый представитель пао сберба...
8,год около часы минута вносить банкомат сбербан...
9,сентябрь год полностью погасить ипотечный кред...
